In [8]:
import deepchem as dc
deepchem.__version__

'2.7.1'

En los tutoriales hasta ahora, hemos seguido un procedimiento simple para entrenar modelos: cargar un conjunto de datos, crear un modelo, llamar a `fit()`, evaluarlo y dar por terminado el proceso. Eso está bien para un ejemplo, pero en proyectos reales de aprendizaje automático, el proceso suele ser más complicado. En este tutorial, examinaremos un flujo de trabajo más realista para entrenar un modelo.


## Optimización de hiperparámetros

Emprezaremos cargando el HIV dataset. Clasifica 40000 molécula en función de si inhiben o no la replicación HIV.

In [9]:
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', split='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

'split' is deprecated.  Use 'splitter' instead.
[10:04:05] WARNING: not removing hydrogen atom without neighbors
[10:04:05] WARNING: not removing hydrogen atom without neighbors
[10:04:37] WARNING: not removing hydrogen atom without neighbors
[10:04:37] WARNING: not removing hydrogen atom without neighbors


Ahora, vamos a entrenar un modelo en estos datos. Utilizaremos un `MultitaskClassifier`, que es simplemente una pila de capas densas. Sin embargo, esto todavía deja muchas opciones abiertas. ¿Cuántas capas debería haber y qué ancho debería tener cada una? ¿Qué tasa de eliminación (dropout) debemos usar? ¿Qué tasa de aprendizaje?

Estos se llaman hiperparámetros. La forma estándar de seleccionarlos es probar muchas combinaciones de valores, entrenar cada modelo en el conjunto de entrenamiento y evaluarlo en el conjunto de validación. Esto nos permite ver cuáles funcionan mejor.

Puedes hacerlo manualmente, pero generalmente es más fácil dejar que la computadora lo haga por ti. **DeepChem proporciona una selección de algoritmos de optimización de hiperparámetros, que se encuentran en el paquete `dc.hyper`**. Para este ejemplo, utilizaremos `GridHyperparamOpt`, que es el método más básico. Simplemente le proporcionamos una lista de opciones para cada hiperparámetro y probará exhaustivamente todas las combinaciones de ellos.

Las listas de opciones se definen mediante un diccionario que proporcionamos. Para cada uno de los argumentos del modelo, proporcionamos una lista de valores para probar. En este ejemplo, consideramos tres posibles conjuntos de capas ocultas: una sola capa de ancho 500, una sola capa de ancho 1000 o dos capas, cada una de ancho 1000. También consideramos dos tasas de eliminación (dropout) (20% y 50%) y dos tasas de aprendizaje (0.001 y 0.0001).


In [14]:
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    #una sola capa de ancho 500, una sola capa de ancho 1000 o dos capas, cada una de ancho 1000. 
    'layer_sizes': [[500], [1000], [1000, 1000]]
}
#Definimos el optimizador que esta dentro de dc.hyper usando GridHyperparamOpt y dentro le indicamos el modelo
optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskClassifier)
#Definimos la métrica de evaluación (ROC)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train_dataset, valid_dataset, metric, transformers)

KeyboardInterrupt: 

**`hyperparam_search()` devuelve tres argumentos: el mejor modelo que encontró, los hiperparámetros para ese modelo y una lista completa de la puntuación de validación para cada modelo.** Echemos un vistazo al último.


In [12]:
all_results

NameError: name 'all_results' is not defined

Podemos observar algunos patrones generales. El uso de dos capas con una tasa de aprendizaje más alta no funciona muy bien. Parece que el modelo más profundo requiere una tasa de aprendizaje más pequeña. También vemos que el dropout del 20% generalmente funciona mejor que el del 50%. Una vez que reducimos la lista de modelos en función de estas observaciones, todas las puntuaciones de validación son muy similares entre sí, probablemente lo suficientemente cercanas como para que la variación restante sea principalmente ruido. No parece marcar una gran diferencia cuál de los conjuntos de hiperparámetros restantes usemos, así que elijamos arbitrariamente una sola capa de ancho 1000 y una tasa de aprendizaje de 0.0001.


## Early Stopping

Existe otro hiperparámetro importante que aún no hemos considerado: cuánto tiempo entrenamos el modelo. `GridHyperparamOpt` entrena cada modelo durante un número fijo de épocas bastante pequeño. Ese número no siempre es el mejor.

Podrías esperar que cuanto más tiempo entrenes, mejor será tu modelo, pero eso no suele ser cierto. Si entrenas durante demasiado tiempo, el modelo generalmente comenzará a ajustarse demasiado a detalles irrelevantes del conjunto de entrenamiento. Puedes darte cuenta de que esto está ocurriendo cuando la puntuación en el conjunto de validación deja de aumentar e incluso puede disminuir, mientras que la puntuación en el conjunto de entrenamiento continúa mejorando.

Afortunadamente, no necesitamos entrenar muchos modelos diferentes durante diferentes números de pasos para identificar el número óptimo. Simplemente lo entrenamos una vez, monitoreamos la puntuación en el conjunto de validación y mantenemos los parámetros que la maximizan. Esto se llama "detención temprana" (early stopping). La clase ValidationCallback de DeepChem puede hacer esto automáticamente por nosotros. En el ejemplo a continuación, le pedimos que calcule el área bajo la curva ROC (ROC AUC) en el conjunto de validación cada 1000 pasos de entrenamiento. Si agregas el argumento save_dir, también guardará una copia de los mejores parámetros del modelo en el disco.


In [26]:
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=0.0001)
#Especificamos un callback que nos vaya avisando cada 1000 pasos la métrica
metric = [dc.metrics.Metric(dc.metrics.roc_auc_score)]
callback = dc.models.ValidationCallback(valid_dataset, 1000, metric)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.74626
Step 2000 validation: roc_auc_score=0.777488
Step 3000 validation: roc_auc_score=0.764931
Step 4000 validation: roc_auc_score=0.76371
Step 5000 validation: roc_auc_score=0.765363
Step 6000 validation: roc_auc_score=0.769145
Step 7000 validation: roc_auc_score=0.771971
Step 8000 validation: roc_auc_score=0.76969
Step 9000 validation: roc_auc_score=0.75637
Step 10000 validation: roc_auc_score=0.762185
Step 11000 validation: roc_auc_score=0.762369
Step 12000 validation: roc_auc_score=0.766334
Step 13000 validation: roc_auc_score=0.767072
Step 14000 validation: roc_auc_score=0.768024
Step 15000 validation: roc_auc_score=0.763817
Step 16000 validation: roc_auc_score=0.765382


0.6580063629150391

## Learning Rate Schedules

En los ejemplos anteriores, utilizamos una tasa de aprendizaje fija durante todo el entrenamiento. En algunos casos, funciona mejor variar la tasa de aprendizaje durante el entrenamiento. Para hacer esto en DeepChem, simplemente especificamos un objeto LearningRateSchedule en lugar de un número para el argumento `learning_rate`. En el siguiente ejemplo, utilizamos una tasa de aprendizaje que disminuye de manera exponencial. Comienza en 0.0002 y luego se multiplica por 0.9 después de cada 1000 pasos.

**NOTA: Una tasa de aprendizaje (Learning_rate) representa la magnitud con la que los parámetros del modelo se actualizan durante el proceso de entrenamiento. En esencia, es un valor que determina cuánto deben cambiar los pesos del modelo en respuesta a los errores calculados durante el entrenamiento.**

In [27]:
'''
Utilizamos una tasa de aprendizaje que disminuye de manera exponencial. Comienza en 0.0002 
y luego se multiplica por 0.9 después de cada 1000 pasos.
'''
learning_rate = dc.models.optimizers.ExponentialDecay(0.0002, 0.9, 1000)
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=learning_rate)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.750672
Step 2000 validation: roc_auc_score=0.774874
Step 3000 validation: roc_auc_score=0.764491
Step 4000 validation: roc_auc_score=0.772914
Step 5000 validation: roc_auc_score=0.769999
Step 6000 validation: roc_auc_score=0.762317
Step 7000 validation: roc_auc_score=0.770688
Step 8000 validation: roc_auc_score=0.770244
Step 9000 validation: roc_auc_score=0.767291
Step 10000 validation: roc_auc_score=0.76709
Step 11000 validation: roc_auc_score=0.765186
Step 12000 validation: roc_auc_score=0.769038
Step 13000 validation: roc_auc_score=0.768695
Step 14000 validation: roc_auc_score=0.770725
Step 15000 validation: roc_auc_score=0.770621
Step 16000 validation: roc_auc_score=0.771401


0.4875274658203125